## Initial configuration
### To start working with this particular notebook, you need to provide necessary credential and settings
### Below is an template of configuration, which is necessary prepare aside of this notebook and copy & paste all content in triple quotes to the next cell's input field
    """
    {
    "ML_MODELS_BUCKET": "ml-saved-models",
    "ML_MODELS_BUCKET_CRN": "xxx",
    "CLOUDANT_API_KEY": "xxx",
    "CLOUDANT_URL": "xxx",
    "CLOUDANT_UI_DB_API_KEY": "xxx",
    "CLOUDANT_UI_DB_URL": "xxx",
    "UTILS_BUCKET": "notebook-utils-bucket",
    "BUCKET_TIFF": "kenya-images",
    "HEIGHTS_TIFF_FILENAME": "WSF3Dv3_Kenya.tif",
    "DB_NAME": "features_db",
    "UI_DB_NAME": "buildings_db",
    "COS_ENDPOINT_URL": "xxx",
    "COS_APIKEY": "xxx",
    "TYPE_SOURCE_FILTER": "area",
    "AREA_TRESHOLD": 0,
    "WML_DEFAULT_SPACE_ID": "xxx",
    "WML_URL": "xxx",
    "WML_APIKEY": "xxx"
    }
    """


In [1]:
import getpass
import json

config_str = getpass.getpass('Enter your prepared config: ')
config = json.loads(config_str)

Enter your prepared config: ········


In [2]:
import ibm_boto3
from botocore.client import Config
import os
import io
import keras
import tensorflow as tf
import tarfile

### Download and prepare selected model

In [ ]:
# define COS connection
data_conn = ibm_boto3.resource(service_name='s3',
    ibm_api_key_id=config["COS_APIKEY"],
    ibm_service_instance_id=config["ML_MODELS_BUCKET_CRN"],
    ibm_auth_endpoint='https://iam.bluemix.net/oidc/token',
    config=Config(signature_version='oauth'),
    endpoint_url=config["COS_ENDPOINT_URL"])

saved_models_bucket_name = config["ML_MODELS_BUCKET"]

saved_models_bucket = data_conn.Bucket(saved_models_bucket_name)
saved_models = [obj.key for obj in saved_models_bucket.objects.all()]
saved_models

In [5]:
# define model name that need to be deployed
model_name = 'Finetuned_Eval_Test_Manual__batchsz32_learnrate0.002-0_epochsFE80_epochstune0_dt08_01_2023_01_21_57.h5'
base_path = os.getcwd()

model_path = os.path.join(base_path, model_name)

fobj = data_conn.Object(saved_models_bucket_name, model_name).get()['Body'].read()
fobj = io.BytesIO(fobj)
with open(model_path, 'wb') as outfile:
    outfile.write(fobj.getbuffer())


In [4]:
os.listdir(base_path)

['Finetuned_Eval_Test_Manual__batchsz32_learnrate0.002-0_epochsFE80_epochstune0_dt08_01_2023_01_21_57.h5']

In [5]:
# zip model into tar.gz file
zip_model_path = os.path.join(base_path, 'model.tar.gz')
tar = tarfile.open(zip_model_path,"w:gz")
tar.add(model_path, arcname='model.h5')
tar.close() 

### Config Watson ML client

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
                    "url": config["WML_URL"],
                    "apikey": config["WML_APIKEY"],
                  }

client = APIClient(wml_credentials)
client.spaces.list()

In [9]:
client.set.default_space(config["WATSON_ML_DEFAULT_SPACE_ID"])

'SUCCESS'

In [10]:
sofware_spec_uid = client.software_specifications.get_id_by_name("runtime-22.2-py3.10")
sofware_spec_uid


'b56101f1-309d-549b-a849-eaa63f77b2fb'

### Deploy the model to the Watson Machine Learning

In [11]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'model_baseline_CFG003',
            client.repository.ModelMetaNames.TYPE: 'tensorflow_2.9',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = client.repository.store_model(
    model=zip_model_path,
    meta_props=metadata)